<a href="https://colab.research.google.com/github/nuno-21902803/ACalculator-android/blob/master/AI_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Relatório AI_1**


    """
        1) We first get the current node by comparing all f cost and selecting the lowest cost node for further expansion
        2) Check max iteration reached or not . Set a message and stop execution
        3) Remove the selected node from yet_to_visit list and add this node to visited list
        4) Perofmr Goal test and return the path else perform below steps
        5) For selected node find out all children (use move to find children)
            a) get the current postion for the selected node (this becomes parent node for the children)
            b) check if a valid position exist (boundary will make few nodes invalid)
            c) if any node is a wall then ignore that
            d) add to valid children node list for the selected parent
            
            For all the children node
                a) if child in visited list then ignore it and try next node
                b) calculate child node g, h and f values
                c) if child in yet_to_visit list then ignore it
                d) else move the child to yet_to_visit list
    """

In [69]:
import pandas as pd
import numpy as np
from types import SimpleNamespace
import logging

In [71]:
df = pd.read_csv("/sample-environment.csv")

df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,S,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,1,F,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,T,F
2,2,F,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
3,3,F,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
4,4,F,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,A,A,F


In [72]:
#dropped 1st column
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,T,F
2,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
3,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
4,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,A,A,F


In [73]:
#converter para uma matrix
matrix = df.values
matrix

array([['S', 'F', 'F', ..., 'F', 'F', 'F'],
       ['F', 'T', 'T', ..., 'A', 'T', 'F'],
       ['F', 'T', 'T', ..., 'A', 'A', 'F'],
       ...,
       ['F', 'T', 'T', ..., 'T', 'T', 'F'],
       ['F', 'T', 'T', ..., 'T', 'T', 'F'],
       ['F', 'F', 'F', ..., 'F', 'F', 'E']], dtype=object)

In [74]:
matrix_heuristic = df.values

for x in range(0, 100, 1):
  for y in range(0, 100, 1):
    if matrix[x,y] == 'A':
      matrix_heuristic[x,y] = 3
    elif matrix[x,y] == 'T':
      matrix_heuristic[x,y] = 1
    elif matrix[x,y] == 'B':
       matrix_heuristic[x,y] = -1
    elif matrix[x,y] == 'S':
       matrix_heuristic[x,y] = 0
    elif matrix[x,y] == 'E':
       matrix_heuristic[x,y] = 0
    else :
       matrix_heuristic[x,y] = -2
      
matrix_heuristic
#onde tem -1 é a barreira, 
#onde tem -2 é a fronteira

array([[0, -2, -2, ..., -2, -2, -2],
       [-2, 1, 1, ..., 3, 1, -2],
       [-2, 1, 1, ..., 3, 3, -2],
       ...,
       [-2, 1, 1, ..., 1, 1, -2],
       [-2, 1, 1, ..., 1, 1, -2],
       [-2, -2, -2, ..., -2, -2, 0]], dtype=object)

In [75]:
import numpy as np

class Node:
    """
        A node class for A* Pathfinding
        parent = parent do current node
        position = posicao do node na matrix
        g = custo desde o inicio
        h = heuristica gerada do current Node ate ao no final
        f = g + h
    """

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position

    def __str__(self):
        return self.position

#Funcao que retorna o path
def return_path(current_node,maze):
    path = []
    no_rows, no_columns = np.shape(maze)
    #colocar -1 em todas as posicoes do maze
    result = [['0' for i in range(no_columns)] for j in range(no_rows)]
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    #Return o path reversed para o que precisarmos do inicio ao fim
    path = path[::-1]
    start_value = 0
    #Update the path com o resultado do A* incrementado com 1
    for i in range(len(path)):
        result[path[i][0]][path[i][1]] = start_value
        start_value += 1
    return result

def search(maze, start, end):
    """
        Return uma lista de tuplos com o path do start e end
        :param maze:
        :param start:
        :param end:
        :return:
    """

    # Cria o node inicial e um node final com valores inicializados
    start_node = Node(None, tuple(start))
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, tuple(end))
    end_node.g = end_node.h = end_node.f = 0

    # Inicializar ambas as listas yet_to_visit e a visited list
    # Nas listas vamos meter todos os noodes que estão yet_to_visit para exploracao. 
    # Depois encontramos o que tem menos custo para expandir depois
    yet_to_visit_list = []  
    #Nesta lista metemos os nodes já visitados para nao explorarmos outra vez
    visited_list = [] 
    
    #start node
    yet_to_visit_list.append(start_node)
    
    #Linhas que vao prevenir loops infinitos 
    outer_iterations = 0
    max_iterations = (len(maze) // 2) ** 10

    #(4 movements) todas as direções

    move  =  [[-1, 0],   # go up
              [0, -1],    # go left
              [1, 0],    # go down
              [0, 1],    # go right
              [-1, -1],
              [-1, 1],
              [1, -1],
              [1, 1]] 

    #Tirar o nºrows e nºcolumns 
    no_rows, no_columns = np.shape(maze)
    
    #Até encontrar ele faz o loop
    
    while len(yet_to_visit_list) > 0:
        
        outer_iterations += 1    

        
        #Node atual
        current_node = yet_to_visit_list[0]
        current_index = 0
        for index, item in enumerate(yet_to_visit_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index
                
        
        #condicao de paragem
        if outer_iterations > max_iterations:
            print ("Stopping pathfinding")
            return return_path(current_node,maze)

        # Retirar current node da  yet_to_visit list, e adiciona na visited list
        yet_to_visit_list.pop(current_index)
        visited_list.append(current_node)

        ##Se node atual == node end entao se sim, retorna o path
        if current_node == end_node:
            return return_path(current_node,maze)

        #Lista dos nodes adjacentes ao current
        children = []

        for new_position in move: 

            # Tirar a node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            #Corresponde às direçoes e esta em range
            if (node_position[0] > (no_rows - 1) or 
                node_position[0] < 0 or 
                node_position[1] > (no_columns -1) or 
                node_position[1] < 0):
                continue

            #Se for barreira/fronteira não passa
            if maze[node_position[0]][node_position[1]] < 0:
                continue

            #Criar node
            new_node = Node(current_node, node_position)

            #E adicionar 
            children.append(new_node)

        #Ver os nodes adjacentes todos

        #inicial conta como o custo maior
        child_tmp= Node()
        child_tmp.g = current_node.g + 3

        for child in children:
            #ver se está na lista de visitados
            if len([visited_child for visited_child in visited_list if visited_child == child]) > 0:
                continue


            child.g = current_node.g + matrix_heuristic[child.position[0], child.position[1]]
            #print(matrix_heuristic[child.position[0], child.position[1]])
            #Criar os f, g, e h valores
            if(child_tmp.g < child.g):
                #print("Anterior" + str(child_tmp.g))
                child = child_tmp
                child.g = child_tmp.g
            else:
                #print("Atual" + str(child.g))              
                child.g = current_node.g + matrix_heuristic[child.position[0], child.position[1]]

            child_tmp = child
            child_tmp.g = child.g

            ##Calcular a heuristica distancia euclediana
            child.h = (((child.position[0] - end_node.position[0]) ** 2) + 
                       ((child.position[1] - end_node.position[1]) ** 2)) 

            child.f = child.g + child.h

            # Se já tiver tsido visitada e o custo for menor
            if len([i for i in yet_to_visit_list if child == i and child.g > i.g]) > 0:
                continue

            # add na lista para ainda explorar yet_to_visit list
            yet_to_visit_list.append(child)
            
            if(child.position[0] == 99 and child.position[1] == 99  and current_node.g > 100):
              print("Final Cost ->" + str(current_node.g))


if __name__ == '__main__':
    start = [0, 0]  # starting position
    end = [99, 99]   # ending position

    path = search(matrix_heuristic, start, end)

    rows = '\n'.join([''.join(["{:" ">3}".format(item) for item in row]) for row in path])
    
    print(rows)
    #pd.DataFrame(path).to_csv("/output_path.csv")

Final Cost ->250
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [83]:
df_copy = pd.read_csv("/sample-environment.csv")
#dropped 1st column
df_copy.drop(columns=df_copy.columns[0], axis=1, inplace=True)
df_copy.head()
matrix_copy_csv = df_copy.values
row_n = 0
column_n = 0

for row in path:
  if row_n == 99:
    row_n = 0
  for colunm in row:
    if int(colunm) > 0:
      matrix_copy_csv[row_n][column_n] = 'SP'
    column_n +=1
  row_n +=1
  column_n = 0

#TODO  acrescentar os 'V' que sao os nos visitados

pd.DataFrame(matrix_copy_csv).to_csv("/output_path.csv")